In [1]:
%reload_ext autoreload
#!pip install -r requirements.txt

In [2]:
%autoreload
from architectures.timegan_cnn_DG import *
from utils import *
from trainers import *
import numpy as np
np.random.seed(455)

trainset = DatasetSinus(num=600, seq_len=100, alpha=0.7, noise=0)
testset = DatasetSinus(num=600, seq_len=100, alpha=0.7, noise=0)
print(len(trainset))
print(trainset[0][0].size())


sin1 freq:[1, 3], phase:[-1.5707963267948966, 0]
sin2 freq:[4, 6], phase:[0, 1.5707963267948966]
sin1 freq:[1, 3], phase:[-1.5707963267948966, 0]
sin2 freq:[4, 6], phase:[0, 1.5707963267948966]
600
torch.Size([100, 3])


In [ ]:
%autoreload
import neptune.new as neptune
import torch

run = neptune.init_run(
    project="kohmann/timeGAN",
    name="timegan_cnn_DG",
    tags=["cnn","DG"],
    description="On updated sinus set",
    source_files=["timegan_cnn_DG.py"],
    capture_hardware_metrics=False,
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI3YjFjNGY5MS1kOWU1LTRmZjgtOTNiYS0yOGI2NDdjZGYzNWUifQ==",
)

device = "cuda" if torch.cuda.is_available() else "cpu"

params = {
    "model_name": "model_cnn_dg.pt",
    "n_epochs": 2000,
    "l_rate": 0.001,
    "l_rate_ae": 0.001,
    "batch_size": 64,
    "num_layers": 2,
    "hidden_dim": 20,
    "Z_dim": 100,
    "dis_thresh": 0.15,
    "dataset": "sinus",
    "feature_dim": trainset[0][0].size(1),
    "max_seq_len": trainset[0][0].size(0),
    "trainset_size": len(trainset),
    "device": device,
    "optimizer": "Adam",
}
run["parameters"] = params

model = TimeGAN(params)
timegan_trainer(model, trainset, params, neptune_logger=run, continue_training=False)

# Generate random synthetic data
gen_z = timegan_generator(model, torch.tensor(testset.T), params["model_name"], torch.device(device), testset[0][0].size(0), 100)

log_visualizations(testset, gen_z, run) # logs pca, tsne, umap, mode_collapse
run["model_checkpoint"].upload(params["model_name"])

run.stop()

https://app.neptune.ai/kohmann/timeGAN/e/TIMEGAN-215
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.

Start Embedding Network Training


Epoch: 499, Loss: 0.0101: 100%|██████████| 500/500 [01:28<00:00,  5.62it/s]



Start Training with Supervised Loss Only


Epoch: 499, Loss: 0.0045: 100%|██████████| 500/500 [01:04<00:00,  7.79it/s]



Start Joint Training


Epoch: 239, E: 0.2318, G: 4.6149, D: 0.1700:  12%|█▏        | 240/2000 [05:05<40:01,  1.36s/it]

In [ ]:
sample = 10
plt.plot(gen_z[sample][:, 0], label="gen_sin1")
plt.plot(gen_z[sample][:, 1], label="gen_sin2")
plt.plot(gen_z[sample][:, 2], label="gen_sin3")
plt.title("Generated sampels")
plt.legend()

In [ ]:
def create_sin3(sin1, sin2, temporal=False):
    sin1, sin2 = np.array(sin1), np.array(sin2)
    e = 0.7  # temporal information weight
    seq_len = len(sin1)
    importance = np.array([e ** i for i in range(seq_len)])

    if temporal:
        sin3 = []
        for i in range(1, seq_len + 1):
            sin3.append(((importance[:i][::-1] * sin1[:i] + importance[:i][::-1] * sin2[:i]) / 2).sum())
        return sin3
    else:
        return [(s1 + s2) / 2 for s1, s2 in zip(sin1, sin2)]

In [ ]:
sample = 6
real_combo = create_sin3(gen_z[sample][:,0], gen_z[sample][:,1], temporal=True)
real_comb_scaled = minmaxscaler().fit_transform(torch.tensor([real_combo]))[0]
gen_combo = gen_z[sample][:,2]
plt.plot(real_comb_scaled, label="true sim3")
plt.plot(gen_combo, label="generated sim3")
plt.title("sim3 - true vs generated")
plt.legend()

In [ ]:
sample = trainset.__getitem__(9)
plt.plot(sample[0][:,0], label="sin1")
plt.plot(sample[0][:,1], label="sin2")
plt.plot(sample[0][:,2], label="(sin1 + sin2)/2")
plt.title("Real data")
plt.legend()

In [ ]:
real_comb = create_sin3(gen_z[sample][:,0], gen_z[sample][:,0], temporal=True)
real_comb_scaled = minmaxscaler().fit_transform(torch.tensor([real_combo]))
plt.plot(real_comb_scaled[0])

### CNN layer testing

In [ ]:
def printLayerShapes(layers, input):
    for layer in layers:
        output = layer(input)

        print(f"{type(layer).__name__:13}: {list(input.size())} --> {list(output.size())}")
        input = output

In [ ]:
import torch
import torch.nn as nn
hidden_dim = 20
max_seq_len = 100
input = torch.randn(2, max_seq_len, hidden_dim)


## Discriminator
layers = nn.Sequential(
            nn.Conv1d(in_channels=100, out_channels=20, kernel_size=(5), stride=2, bias=False)
            , nn.BatchNorm1d(20)
            , nn.LeakyReLU()
            , nn.Conv1d(in_channels=20, out_channels=40, kernel_size=(7), stride=2, bias=False)
            , nn.BatchNorm1d(40)
            , nn.LeakyReLU()
            , nn.Flatten(start_dim=1)
            , nn.Linear(40, 1)
        )
print(f"Discriminator architecture\nInput: {input.size()}\n")
printLayerShapes(layers, input)

In [ ]:
## Generator architecture
Z_dim = 100
hidden_dim = 20
max_seq_len = 100
input = torch.randn(2, max_seq_len, Z_dim)


layers = nn.Sequential(
            nn.Conv1d(in_channels=100, out_channels=100, kernel_size=(6), stride=2, bias=False)
            , nn.BatchNorm1d(100)
            , nn.LeakyReLU()
            , nn.Conv1d(in_channels=100, out_channels=100, kernel_size=(9), stride=2, bias=True)
            , nn.Sigmoid()
        )

print(f"Generator architecture\nInput: {input.size()}\n")
printLayerShapes(layers, input)

In [ ]:
## Generator architecture LINEAR
Z_dim = 1
hidden_dim = 20
max_seq_len = 100
input = torch.randn(16, max_seq_len, Z_dim)
input = input.squeeze(dim=2)


layers = nn.Sequential(
            nn.Linear(in_features=100, out_features=200, bias=True)
            ,nn.LayerNorm(200)
            ,nn.LeakyReLU()
            ,nn.Linear(in_features=200, out_features=400, bias=True)
            ,nn.LayerNorm(400)
            ,nn.LeakyReLU()
            ,nn.Linear(in_features=200, out_features=1000, bias=True)
            ,nn.LayerNorm(400)
            ,nn.LeakyReLU()
        )


print(f"Generator architecture\nInput: {input.size()}\n")
printLayerShapes(layers, input)

In [ ]:
torch.randn(16,300).view(-1,100,3).size()